# Analyzing Spectral Reflectance within Intertidal Zones

**Summary**
This tutorial demonstrates how to use Earth observation data from NASA's ECOSTRESS and EMIT missions alongside in situ spectral reflectance measurements to investigate environmental conditions and biodiversity in the intertidal zones of Southern California. Specifically, this notebook focuses on analyzing the variability of shorelines by analyzing the coefficient of variation and testing the hypothesis within the interridal zones: Greater Coefficient of Variation = Greater Variability. 

We aim to better understand how spectral signatures correspond to observable biological patterns by integrating:

- EMIT full-band Level 2A reflectance data, 

- EMIT full-band level 2A mask data, and

- Normalized field-collected spectral data

Ultimately, the goal is to support coastal monitoring and climate adaptation efforts by incorporating various NASA missions, with EMIT as the first trial.

This tutorial assumes that you have three sites since it was tested on three - Newport Beach, Palos Verdes, and San Diego

**Learning Objectives**

- What is spectral diversity and how does it relate to biodiversity?
- Does a greater coefficient of variation = more variability within the intertidal zones?
- How to prepare and analyze reflectance data sets from on-ground and satellite observations for various project goals. 
- Compare spectral diversity across sites and sensor types using summary statistics and visual plots.

**This Code Assumes You Have:**
- One normalized CSV for field reflectance - in situ data
- One .geojson polygon file for ROI / Specific Points - Created through coordinates or a polygon mapped in QGIS/ArcGIS
- ECOSTRESS and EMIT granules .txt file from VITALS Tutorial 1

**Relevant Links**
- Process data for analysis: http://github.com/kyla-mbc/Intertidal/blob/main/JupyterScript/DataProcess.ipynb
- To process .sed data from instrument to csv: https://github.com/kyla-mbc/Intertidal/blob/main/PythonCode/SED.py
- To normalize raw spectra csv: https://github.com/kyla-mbc/Intertidal/blob/main/PythonCode/SED.py
- To manually download EMIT / ECOSTRESS Data: https://ecostress.jpl.nasa.gov/downloads/tutorials/06-Downloading_from_AppEEARS.pdf
- This Jupyter Notebook would not be possible without the help and structure of the VITALS tutorials to find and analyze concurrent ECOSTRESS and EMIT data: https://github.com/nasa/VITALS/tree/main


_______________________________________________________________________________________________________________________________________________________

## Part 1: Data Setup and Dependencies

Create a working environment with required packages. Be sure to active your environment and choose the right kernel each time you work on your project.

In [ ]:
#Run in Terminal: mamba create -n (environmentname) -c conda-forge --yes python=3.12 gdal fiona hvplot geoviews rioxarray rasterio jupyter geopandas earthaccess jupyter_bookeh h5py h5netcdf spectral sckit-image jupyterlab seaborn dask ray-default pystac-client odc-stac pyresample libgal-hdf4 harmony-py

# Setup & Imports
import warnings
warnings.filterwarnings("ignore")

# Import required libraries
import os
import json
import folium
import earthaccess
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import rioxarray as rxr
import requests
import math
from osgeo import gdal
import rasterio as rio
import hvplot.xarray
import hvplot.pandas
import holoviews as hv

from branca.element import Figure
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.geometry.polygon import orient
from datetime import timedelta
from earthaccess import DataGranules
from pathlib import Path
from tqdm.notebook import tqdm
from modules.emit_tools import emit_xarray, spatial_subset, ortho_xr
from matplotlib.backends.backend_pdf import PdfPages


%matplotlib inline


### Part 1a: NASA Earthdata Authentication

To access data from NASA Earthdata, you must first authenticate using the earthaccess library. Follow the interactive login prompt that appears to sign in with your NASA Earthdata credentials (email and password associated with your Earthdata account). This step ensures you have authorized access to download and search Earth observation datasets.

If you haven’t registered yet, create a free account here: https://urs.earthdata.nasa.gov/users/new

In [ ]:
# Authenticate with NASA Earthdata
auth = earthaccess.Auth()
auth.login(strategy="interactive", persist=True)
print('Authenticated:', auth.authenticated)

### Part 1b: Define Paths 
In order to put all the data together (ECOSTRESS, EMIT, in situ), it is helpful to know where they are all located and assigning them to variables will help especially with longer file names. To download data for concurrent ECOSTRESS and EMIT data, refer to DataProcess.ipynb jupyter notebook.

You may not use all the paths or all of them at once, but it is good to have them organized. 

In [ ]:
# Define Paths
#Base Path
base_dir = Path("...") # Common folder for all inputs and outputs. 

#In Situ Information
#Normalized csv files
LCDMground_truth_csv = Path(".../LCDMNormalized.csv") #Normalized in situ csv for analysis.
SDground_truth_csv = Path(".../SDNormalized.csv")
PVground_truth_csv = Path(".../PVNormalized.csv")

#ROI Polygons 
LCDMPolygon = gpd.read_file("....../LCDMPolygon.geojson")
SDPolygon = gpd.read_file("....../SDPolygon.geojson")
PVPolygon = gpd.read_file("....../PVPolygon.geojson")

#ROI Quadrats in situ
LCDMQuadrat = gpd.read_file("....../LCDMQuadrat.geojson") 
SDQuadrat = gpd.read_file("....../SDQuadrat.geojson")
PVQuadrat = gpd.read_file("....../PVQuadrat.geojson")

#if only using one site, use these variables / paths instead. 
# roi_path = Path("....../LCDMPolygon.geojson") #Region of Interest - Polygon
# roi_point=Path("....../LCDMQuadrat.geojson") #Region of Interest - Quadrat Points

#ecostress and emit data - LCDM 
ecostress_tif_pathLCDM  = Path("....../ECOSTRESS") #ECOSTRESS .tif folder (optional if not using earthdata)
emit_fp_LCDM = "....../EMIT_L2A_RFL_001_20250525T213443_2514514_003.nc"
qa_fp_LCDM = "....../EMIT_L2A_MASK_001_20250525T213443_2514514_003.nc"

#ecostress and emit data - San Diego
ecostress_tif_pathSD = Path(".../data/SD/ECOSTRESS") #ECOSTRESS .tif folder (optional if not using earthdata)
emit_fp_SD = ".../data/SD/EMIT_L2A_RFL_001_20250626T164730_2517711_009.nc"
qa_fp_SD = ".../data/SD/EMIT_L2A_MASK_001_20250626T164730_2517711_009.nc"

#ecostress and emit data - Palos Verdes
ecostress_tif_pathPV = Path(".../data/PV/ECOSTRESS") #ECOSTRESS .tif folder (optional if not using earthdata)
emit_fp_PV = ".../data/PV/EMIT_L2A_RFL_001_20250525T213443_2514514_003.nc"
qa_fp_PV = ".../data/PV/EMIT_L2A_MASK_001_20250525T213443_2514514_003.nc"

# ecostress_tif_path.mkdir(exist_ok=True) #make directory if it does not exist. 


## Part 2: Visualizing Regions of Interest
It is helpful to validate whether your data was loaded in properly. Visualizing your AOI will also assist in analysis. You can either view this as a plot or through an interactive map.

In [ ]:
#Visualize Sample Locations on a Plot.
# Load GeoJSON files
print(LCDMQuadrat.head())
print(SDQuadrat.head())
print(PVQuadrat.head())

# Set up the plot with Cartopy for mapping
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Add high-resolution coastlines and gridlines
ax.coastlines(resolution='10m')
gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
gl.top_labels = gl.right_labels = False

# Plot the GeoDataFrames
LCDMQuadrat.plot(ax=ax, marker='o', color='red', label='LCDM', transform=ccrs.PlateCarree())
SDQuadrat.plot(ax=ax, marker='^', color='green', label='SD', transform=ccrs.PlateCarree())
PVQuadrat.plot(ax=ax, marker='s', color='blue', label='PV', transform=ccrs.PlateCarree())

plt.legend()
plt.title("Site Sampling Locations")
plt.show()


# Visualize all locations within one grid.
def plot_quadrats(gdf, title, color, marker):
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    
    ax.coastlines(resolution='10m')    # Plot coastlines
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)     # Gridlines
    gl.top_labels = gl.right_labels = False
    gdf.plot(ax=ax, color=color, marker=marker, alpha=0.8, transform=ccrs.PlateCarree(), label=title)     # Plot quadrats

    # Auto-zoom to the bounds of the GeoDataFrame (pad for clarity)
    minx, miny, maxx, maxy = gdf.total_bounds
    padding = 0.0001 # degree buffer
    ax.set_extent([minx - padding, maxx + padding, miny - padding, maxy + padding], crs=ccrs.PlateCarree())

    # Title and legend
    plt.title(title)
    plt.legend()
    plt.show()


# Visualize each location separately
#Coasts will not be visible due to space constraints.
plot_quadrats(LCDMQuadrat, "LCDM Quadrat Sites", "red", "o") 
plot_quadrats(SDQuadrat,   "SD Quadrat Sites",   "green", "^")
plot_quadrats(PVQuadrat,   "PV Quadrat Sites",   "blue", "s")


Alternatively, you could also plot your ROIs as well as the quadrats within them through an interactive map. 

On the top right corner of the map, you can toggle on and off the various layers of the map.

In [ ]:
#Visualize Sample Locations on an Interactive Map.

# Load all 3 ROI polygon GeoJSONs
LCDMpolygon = gpd.read_file("....../LCDMPolygon.geojson")
SDpolygon   = gpd.read_file("....../SDPolygon.geojson")
PVpolygon   = gpd.read_file("....../PVPolygon.geojson")

#Load points within locations
LCDM_points = gpd.read_file("....../LCDMQuadrat.geojson")
SD_points   = gpd.read_file("....../SDQuadrat.geojson")
PV_points   = gpd.read_file("....../PVQuadrat.geojson")


# Combine to get a universal bounding box
combined = gpd.GeoDataFrame(pd.concat([LCDMpolygon, SDpolygon, PVpolygon], ignore_index=True), crs=LCDMpolygon.crs)
bbox = combined.total_bounds  # [minx, miny, maxx, maxy]
agu_bbox = gpd.GeoDataFrame(geometry=[combined.unary_union.envelope], crs=combined.crs)

# Create interactive map using folium and Google Satellite basemap
fig_roi = Figure(width="750px", height="450px")
map_roi = folium.Map(
    tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', 
    attr='Google Satellite', 
    name='Google Satellite'
)
fig_roi.add_child(map_roi)

# Add bounding box layer
folium.GeoJson(
    agu_bbox,
    name='Bounding Box'
).add_to(map_roi)

# Add all 3 polygons to the map with different colors
LCDMpolygon.explore(
    m=map_roi,
    name="LCDM",
    column=None,
    style_kwds=dict(color="red", fillOpacity=0.1, weight=10),
    popup=True
)

SDpolygon.explore(
    m=map_roi,
    name="SD",
    column=None,
    style_kwds=dict(color="green", fillOpacity=0.1, weight=10),
    popup=True
)

PVpolygon.explore(
    m=map_roi,
    name="PV",
    column=None,
    style_kwds=dict(color="blue", fillOpacity=0.1, weight=10),
    popup=True
)

# Add LCDM points
for _, row in LCDM_points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        popup=row.get("name", "LCDM Point")
    ).add_to(map_roi)

# Add SD points
for _, row in SD_points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="green",
        fill=True,
        fill_color="green",
        popup=row.get("name", "SD Point")
    ).add_to(map_roi)

# Add PV points
for _, row in PV_points.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        popup=row.get("name", "PV Point")
    ).add_to(map_roi)



# Add layer control toggle
map_roi.add_child(folium.LayerControl())

# Fit map bounds to the union of all 3 polygons
minx, miny, maxx, maxy = combined.total_bounds
map_roi.fit_bounds(bounds=((miny, minx), (maxy, maxx)))

# Show map
fig_roi


## Part 3: Load In Situ Data

Load in and preview the normalized in situ data set. 


In [ ]:
from pathlib import Path
import pandas as pd

def load_ground_truth_csv(csv_path):
    """
    Loads a ground truth CSV file, drops columns 2–6, and displays the first few rows.

    Parameters:
        csv_path (str or Path): Path to the CSV file.

    Returns:
        pd.DataFrame or None: Returns the cleaned DataFrame if loaded, or None if file doesn't exist.
    """
    csv_path = Path(csv_path)  # Ensure it's a Path object
    if csv_path.exists():
        df = pd.read_csv(csv_path)

        # Drop columns 2–6 (index 1 to 5)
        df.drop(df.columns[1:6], axis=1, inplace=True)

        print(f"Loaded and trimmed file: {csv_path.name}")
        display(df.head())
        return df
    else:
        print(f"File not found: {csv_path.name}")
        return None


def load_ground_truth_csv(csv_path):
    """
    Loads a ground truth CSV file, drops columns 2–6, and displays the first few rows.

    Parameters:
        csv_path (str or Path): Path to the CSV file.

    Returns:
        pd.DataFrame or None: Returns the cleaned DataFrame if loaded, or None if file doesn't exist.
    """
    csv_path = Path(csv_path)  # Ensure it's a Path object
    if csv_path.exists():
        df = pd.read_csv(csv_path)

        # Drop columns 2–6 (index 1 to 5)
        df.drop(df.columns[1:6], axis=1, inplace=True)

        print(f"Loaded file: {csv_path.name}")
        display(df.head())
        return df
    else:
        print(f"File not found: {csv_path.name}")
        return None


In [ ]:
# Load and preview each location's normalized in situ data
LCDM_df = load_ground_truth_csv(LCDMground_truth_csv)
SD_df= load_ground_truth_csv(SDground_truth_csv)
PV_df = load_ground_truth_csv(PVground_truth_csv)

### Part 3a: Organizing Data 

Depending on how you took your in situ data, you would want to organize your data well to make processing easier. This part of the code will:

- Melt your wide format into a long format
- Define transects and quadrats for your data. (Useful if you do not have "perfect" data sets.)

In [ ]:
#Melting function to change the format of your in situ data. 
def melt_in_situ_df(df, site_name):
    """
    Converts a wide-format in situ reflectance DataFrame into long format.
    
    Parameters:
        df (pd.DataFrame): Raw wide-format DataFrame (columns: wavelength, take_1, take_2, ...)
        site_name (str): Name of the site to tag the data
    
    Returns:
        pd.DataFrame: Long-format DataFrame with columns: site, wavelength, take_id, reflectance
    """
    if df is None:
        return pd.DataFrame()  # skip empty sites
    
    melted = df.melt(id_vars=["wavelength"], var_name="take_id", value_name="reflectance")
    melted["site"] = site_name
    return melted


In [ ]:
#Apply the functions
LCDM_melt = melt_in_situ_df(LCDM_df, "LCDM")
SD_melt = melt_in_situ_df(SD_df, "SD")
PV_melt = melt_in_situ_df(PV_df, "PV")

In [ ]:
# Concatenate all melted data
long_df = pd.concat([LCDM_melt, SD_melt, PV_melt], ignore_index=True)
long_df.head()


 Not all samples are pefect, if you need to sort out transects and quadrats, follow the code below.

In [ ]:
# Define take-to-quadrat mapping
def build_quadrat_map(take_ids, n_quadrats, custom_take_counts=None):
    """
    Build a mapping from take_id → quadrat, taking into account custom take counts.
    """
    mapping = []
    idx = 0
    for q in range(1, n_quadrats + 1):
        count = custom_take_counts.get(q, 3) if custom_take_counts else 3
        for _ in range(count):
            if idx >= len(take_ids):
                break
            mapping.append({'take_id': take_ids[idx], 'quadrat': q})
            idx += 1
    return pd.DataFrame(mapping)

# Add transect number based on quadrat
def assign_transects(quadrat_map, quadrats_per_transect=6):
    """
    Adds a transect number to the quadrat mapping DataFrame.
    """
    quadrat_map['transect'] = ((quadrat_map['quadrat'] - 1) // quadrats_per_transect) + 1
    return quadrat_map


In [ ]:
#Define custom quadrats by transect
def build_quadrat_from_transects(transect_quadrat_counts):
    """
    Given a dict like {1: 6, 2: 6, 3: 4}, returns a list of dicts:
    [{'quadrat': 1, 'transect': 1}, {'quadrat': 2, 'transect': 1}, ...]
    """
    metadata = []
    quadrat_counter = 1
    for transect, n_quadrats in transect_quadrat_counts.items():
        for _ in range(n_quadrats):
            metadata.append({'quadrat': quadrat_counter, 'transect': transect})
            quadrat_counter += 1
    return pd.DataFrame(metadata)


In [ ]:
#Layouts for # of quadrats within each transect
LCDM_layout = {
    1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6,
    7: 6, 8: 6, 9: 6,
    10: 6
}

# PV layout: transects 1–6 have 6 quadrats, 7–9 have 4, transect 10 has 5
PV_layout = {
    1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6,
    7: 4, 8: 4, 9: 3,
    10: 5
}

# SD layout:
SD_layout = {
    1: 3, 2: 3,
    3: 3, 4: 5, 5: 6, 6: 6, 7: 6,
    8: 5, 9: 6, 10: 6, 11: 6
}


In [ ]:
def build_quadrat_map_with_custom_layout(take_ids, layout_dict, custom_take_counts=None):
    """
    Maps take_ids to quadrats and transects based on layout_dict.
    """
    quadrat_meta = build_quadrat_from_transects(layout_dict)
    
    mapping = []
    idx = 0
    for row in quadrat_meta.itertuples():
        count = custom_take_counts.get(row.quadrat, 3) if custom_take_counts else 3
        for _ in range(count):
            if idx >= len(take_ids):
                break
            mapping.append({
                'take_id': take_ids[idx],
                'quadrat': row.quadrat,
                'transect': row.transect
            })
            idx += 1
    return pd.DataFrame(mapping)


In [ ]:
# Get sorted take_ids
PV_take_ids = PV_melt['take_id'].dropna().sort_values().unique().tolist()
SD_take_ids = SD_melt['take_id'].dropna().sort_values().unique().tolist()
LCDM_take_ids = LCDM_melt['take_id'].dropna().sort_values().unique().tolist()

# Custom takes if any
LCDM_custom = {41: 2, 53: 2, 54: 2}
SD_custom = {}  
PV_custom = {}  

# Build mapping tables
LCDM_map = build_quadrat_map_with_custom_layout(LCDM_take_ids, LCDM_layout, LCDM_custom)
SD_map = build_quadrat_map_with_custom_layout(SD_take_ids, SD_layout, SD_custom)
PV_map = build_quadrat_map_with_custom_layout(PV_take_ids, PV_layout, PV_custom)

# Merge into full datasets
LCDM_sorted = LCDM_melt.merge(LCDM_map, on="take_id", how="left")
SD_sorted = SD_melt.merge(SD_map, on="take_id", how="left")
PV_sorted = PV_melt.merge(PV_map, on="take_id", how="left")


In [ ]:
#Ensure transects and quadrats print as integers
LCDM_sorted['quadrat'] = LCDM_sorted['quadrat'].astype('Int64')
LCDM_sorted['transect'] = LCDM_sorted['transect'].astype('Int64')

SD_sorted['quadrat'] = SD_sorted['quadrat'].astype('Int64')
SD_sorted['transect'] = SD_sorted['transect'].astype('Int64')

PV_sorted['quadrat'] = PV_sorted['quadrat'].astype('Int64')
PV_sorted['transect'] = PV_sorted['transect'].astype('Int64')


In [ ]:
#Check if code to sort transects and quadrats was a success.
print("LCDM: Quadrats =", LCDM_sorted['quadrat'].nunique(), 
      " | Transects =", LCDM_sorted['transect'].nunique())

print("SD: Quadrats =", SD_sorted['quadrat'].nunique(), 
      " | Transects =", SD_sorted['transect'].nunique())

print("PV: Quadrats =", PV_sorted['quadrat'].nunique(), 
      " | Transects =", PV_sorted['transect'].nunique())


print("🔍 LCDM Sorted Preview")
display(LCDM_sorted.head())

print("🔍 SD Sorted Preview")
display(SD_sorted.head())

print("🔍 PV Sorted Preview")
display(PV_sorted.head())


For Data Interpolation - change nm to omit odd features.
This is not required code, but can be used to interpolate your data for reasons such as instrument error or to minimize data analysis.

In [ ]:
# def interpolate_to_5nm(df, value_col='reflectance'):
#     new_wavelengths = np.arange(400, 910, 10)
#     interpolated = []

#     for take_id, group in df.groupby('take_id'):
#         group = group.sort_values('wavelength')
#         interp_vals = np.interp(new_wavelengths, group['wavelength'], group[value_col])

#         new_group = pd.DataFrame({
#             'take_id': take_id,
#             'wavelength': new_wavelengths,
#             value_col: interp_vals,
#         })

#         # Optional: carry over metadata
#         for col in ['site', 'transect', 'quadrat']:
#             if col in group.columns:
#                 new_group[col] = group[col].iloc[0]

#         interpolated.append(new_group)

#     return pd.concat(interpolated, ignore_index=True)


In [ ]:
# LCDM_interp = interpolate_to_5nm(LCDM_sorted)

# LCDM_interp.groupby('take_id')['wavelength'].count().value_counts().sort_index()
# print(LCDM_interp['wavelength'].unique())


## Part 4: Calculate In Situ Data

Calculate for the mean, std, and coefficient of variation for the in situ data. 

In [ ]:
#Function to calculate statistics for in situ data. 
def summarize_in_situ_data(df):
    summary = (
        df
        .groupby(['site', 'transect', 'quadrat', 'wavelength'])
        .agg(
            mean_reflectance=('reflectance', 'mean'),
            std_reflectance=('reflectance', 'std')
        )
        .reset_index()
    )
    summary['cv_percent'] = (summary['std_reflectance'] / summary['mean_reflectance']) * 100
    return summary


In [ ]:
#Apply to all in situ field sites. 
LCDM_summary = summarize_in_situ_data(LCDM_sorted)
SD_summary   = summarize_in_situ_data(SD_sorted)
PV_summary   = summarize_in_situ_data(PV_sorted)


In [ ]:
#OPTIONAL: You can download the csv files with the outputted calculations. 
LCDM_summary.to_csv(".../IntertidalOutputs/LCDM_summary.csv", index=False)
SD_summary.to_csv(".../IntertidalOutputs/SD_summary.csv", index=False)
PV_summary.to_csv(".../IntertidalOutputs/PV_summary.csv", index=False)


In [ ]:
#Combine all the data
all_summary = pd.concat([LCDM_summary, SD_summary, PV_summary], ignore_index=True)
all_summary.to_csv("all_sites_summary.csv", index=False)

## Part 5: Plot In Situ Spectra


### With the in situ data loaded in, you can plot the data by site.

Visualize the standard deviation and coefficient of variation for the different sites. Plotting all the points for a site can get quite messy, so it is best to divide them up into either transects or quadrats, depending on your analysis. 

In [ ]:
import matplotlib.pyplot as plt

def plot_transect_spectra_from_summary(df, site_name="LCDM", save_path=None):
    """
    Plots reflectance spectra for each transect in the summary DataFrame.

    Parameters:
        df: DataFrame with 'transect', 'wavelength', and 'mean_reflectance' columns
        site_name: str, site label
        save_path: str, optional PDF file path
    """
    plt.figure(figsize=(8, 5))

    for transect_id in sorted(df['transect'].unique()):
        sub = df[df['transect'] == transect_id]
        plt.plot(
            sub['wavelength'],
            sub['mean_reflectance'],
            label=f'Transect {transect_id}',
            linewidth=1.5
        )

    plt.title(f"{site_name} – Mean Reflectance per Transect", fontsize=12)
    plt.xlabel("Wavelength (nm)", fontsize=10)
    plt.ylabel("Mean Reflectance", fontsize=10)
    plt.xlim(400, 900)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend(title="Transect", fontsize=8, title_fontsize=9)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")

    plt.show()


In [ ]:
#Plot out spectra and to pdf.
plot_transect_spectra_from_summary(LCDM_summary, "LCDM", save_path=".../IntertidalOutputs/LCDM_allstats.pdf")
plot_transect_spectra_from_summary(SD_summary, "SD", save_path=".../IntertidalOutputs/SD_sallstats.pdf")
plot_transect_spectra_from_summary(PV_summary, "PV", save_path=".../IntertidalOutputs/PV_allstats.pdf")


The graphs can get really messy, so plotting the averages of each transect per site is very helpful as well as visualizing the mean and standard deviation on different graphs.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_transect_spectra(summary_df, site_name, save_path=None):
    """
    Plots mean reflectance spectra, CV, and standard deviation per transect.

    Parameters:
        summary_df (pd.DataFrame): DataFrame containing 'site', 'transect', 'wavelength', and 'mean_reflectance'
        site_name (str): Name of site ("LCDM", "SD", or "PV")
        save_path (str): If provided, saves the figure to this path
    """
    # Filter to only the selected site
    site_df = summary_df[summary_df['site'] == site_name]

    # Group by transect and wavelength
    grouped = site_df.groupby(['transect', 'wavelength'])['mean_reflectance'] # <-- Change to reflectance for interpolated data. 


    # Compute all metrics
    stats = grouped.agg(['mean', 'std']).reset_index()
    stats['cv'] = stats['std'] / stats['mean']

    fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

    for transect in sorted(stats['transect'].unique()):
        sub = stats[stats['transect'] == transect]
        axes[0].plot(sub['wavelength'], sub['mean'], label=f'Transect {transect}')
        axes[1].plot(sub['wavelength'], sub['std'], label=f'Transect {transect}')
        axes[2].plot(sub['wavelength'], sub['cv'], label=f'Transect {transect}')

    axes[0].set_title(f"{site_name} – Mean Reflectance per Transect")
    axes[0].set_ylabel("Mean Reflectance")

    axes[1].set_title(f"{site_name} – Standard Deviation per Transect")
    axes[1].set_ylabel("Standard Deviation")

    axes[2].set_title(f"{site_name} – Coefficient of Variation per Transect")
    axes[2].set_xlabel("Wavelength (nm)")
    axes[2].set_ylabel("CV (std/mean)")

    for ax in axes:
        ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
        ax.grid(True)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)

    plt.show()
    plt.close()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_transect_spectra(summary_df, site_name, save_path=None):
    """
    Plots mean reflectance and standard deviation per transect.

    Parameters:
        summary_df (pd.DataFrame): DataFrame containing 'site', 'transect', 'wavelength', and 'mean_reflectance'
        site_name (str): Name of site ("LCDM", "SD", or "PV")
        save_path (str): If provided, saves the figure to this path
    """
    # Filter to only the selected site
    site_df = summary_df[summary_df['site'] == site_name]

    # Group by transect and wavelength
    grouped = site_df.groupby(['transect', 'wavelength'])['mean_reflectance']

    # Compute mean and standard deviation
    stats = grouped.agg(['mean', 'std']).reset_index()

    fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    for transect in sorted(stats['transect'].unique()):
        sub = stats[stats['transect'] == transect]
        axes[0].plot(sub['wavelength'], sub['mean'], label=f'Transect {transect}')
        axes[1].plot(sub['wavelength'], sub['std'], label=f'Transect {transect}')

    axes[0].set_title(f"{site_name} – Mean Reflectance per Transect")
    axes[0].set_ylabel("Mean Reflectance")

    axes[1].set_title(f"{site_name} – Standard Deviation per Transect")
    axes[1].set_xlabel("Wavelength (nm)")
    axes[1].set_ylabel("Standard Deviation")

    for ax in axes:
        ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
        ax.grid(True)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved to: {save_path}")

    plt.show()
    plt.close()


Interpolated Data Charts

In [ ]:
# plot_transect_spectra(LCDM_interp, "LCDM", save_path="LCDM_transects.pdf")
# plot_transect_spectra(SD_interp, "SD", save_path="SD_transects.pdf")
# plot_transect_spectra(PV_interp, "PV", save_path="PV_transects.pdf")

In [ ]:
#Plot out spectra and to pdf.
plot_transect_spectra(LCDM_summary, "LCDM", save_path=".../IntertidalOutputs/LCDM_allstats.pdf")
plot_transect_spectra(SD_summary, "SD", save_path=".../IntertidalOutputs/SD_sallstats.pdf")
plot_transect_spectra(PV_summary, "PV", save_path=".../IntertidalOutputs/PV_allstats.pdf")


Below is another way to plot your spectra and standard deviation.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_mean_std_spectra(summary_df, site_name, save_path=None):
    """
    Plots mean reflectance and standard deviation (as shaded area) per transect.

    Parameters:
        summary_df (pd.DataFrame): DataFrame with columns ['site', 'transect', 'wavelength', 'mean_reflectance']
        site_name (str): Name of the site to filter by (e.g., "LCDM", "SD", "PV")
        save_path (str): Path to save the plot as PDF (optional)
    """
    # Filter by site
    site_df = summary_df[summary_df['site'] == site_name]

    # Group and compute stats
    grouped = site_df.groupby(['transect', 'wavelength'])['mean_reflectance']
    stats = grouped.agg(['mean', 'std']).reset_index()

    # Adjust figure size to reduce horizontal padding
    fig, ax = plt.subplots(figsize=(10, 6))  

    for transect in sorted(stats['transect'].unique()):
        sub = stats[stats['transect'] == transect]
        ax.plot(sub['wavelength'], sub['mean'], label=f'Transect {transect}')
        ax.fill_between(
            sub['wavelength'],
            sub['mean'] - sub['std'],
            sub['mean'] + sub['std'],
            alpha=0.2
        )

    ax.set_title(f"{site_name} Site – Mean In Situ Reflectance per Transect")
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Reflectance")
    ax.legend(title="Transect", loc='upper left', ncol=2, fontsize='small')
    ax.grid(True)

    # Reduce outer margins
    plt.tight_layout(pad=1.5)

    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"Saved to: {save_path}")

    plt.show()
    plt.close()


In [ ]:
plot_mean_std_spectra(summary_df=LCDM_summary, site_name="LCDM", save_path="LCDM_InSitu_Reflectance.pdf")

For interpolated Data. 

In [ ]:
# # Rename 'reflectance' to match expected column name
# LCDM_interp = LCDM_interp.rename(columns={"reflectance": "mean_reflectance"})

# # Add 'site' column (if not already there)
# LCDM_interp['site'] = "LCDM"

# # Combine into one summary DataFrame (optional if you want to reuse one function call)
# summary_dfLCDM = pd.concat([LCDM_interp], ignore_index=True)

# # Plot and save each one separately
# plot_mean_std_spectra(summary_dfLCDM, site_name="LCDM", save_path="LCDM_transectsInSItu.pdf")



### You can also plot your in situ data by transect.

In [ ]:
#PLot by transect
import matplotlib.pyplot as plt

def plot_all_transects_per_site(summary_df, site_name, show=True, save=False, output_dir=".", file_format="png"):
    """
    Plots one figure per transect in the specified site. Each plot shows all quadrats in that transect.
    
    Parameters:
        summary_df (pd.DataFrame): Summary data with site, transect, quadrat, wavelength, mean_reflectance
        site_name (str): e.g., "LCDM", "SD", "PV"
        show (bool): If True, display plots
        save (bool): If True, save each figure as a file
        output_dir (str): Folder to save plots in (if save=True)
        file_format (str): "png", "pdf", etc.
    """
    site_df = summary_df[summary_df['site'] == site_name]
    transects = sorted(site_df['transect'].unique())
    
    for transect in transects:
        df = site_df[site_df['transect'] == transect]
        quadrats = sorted(df['quadrat'].unique())

        plt.figure(figsize=(10, 6))
        for q in quadrats:
            sub = df[df['quadrat'] == q]
            # Multiply reflectance by 100 to convert to percentage
            plt.plot(sub['wavelength'], sub['mean_reflectance'] * 100, label=f"Q{q}")

        plt.title(f"{site_name} – Transect {transect}")
        plt.xlabel("Wavelength (nm)")
        plt.ylabel("Mean Reflectance (%)")  # Update label to show percentage
        plt.legend(loc='upper left', fontsize='small', ncol=2)
        plt.grid(True)
        plt.tight_layout()
        
        if save:
            filename = f"{site_name}_Transect{transect}.{file_format}"
            filepath = f"{output_dir}/{filename}"
            plt.savefig(filepath, dpi=300)
        
        if show:
            plt.show()
        else:
            plt.close()

In [ ]:
# Display the plots by quadrat
plot_all_transects_per_site(LCDM_summary, "LCDM")
plot_all_transects_per_site(SD_summary, "SD")
plot_all_transects_per_site(PV_summary, "PV")

In [ ]:
# import matplotlib.pyplot as plt
# import os

# def plot_all_transects_per_site(summary_df, site_name, show=True, save=True, output_dir="plots", file_format="png"):
#     """
#     Plots and optionally saves one figure per transect in the specified site.
#     Each figure shows the spectral reflectance of all quadrats in that transect.

#     Parameters:
#         summary_df (pd.DataFrame): Summary data with columns: site, transect, quadrat, wavelength, mean_reflectance
#         site_name (str): e.g., "LCDM", "SD", "PV"
#         show (bool): If True, display plots
#         save (bool): If True, save each figure to file
#         output_dir (str): Directory to save plots (created if doesn't exist)
#         file_format (str): File type (e.g., "png", "pdf")
#     """
#     df_site = summary_df[summary_df['site'] == site_name]
#     transects = sorted(df_site['transect'].unique())

#     if save and not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for transect in transects:
#         df_transect = df_site[df_site['transect'] == transect]
#         quadrats = sorted(df_transect['quadrat'].unique())

#         plt.figure(figsize=(10, 6))
#         for q in quadrats:
#             df_q = df_transect[df_transect['quadrat'] == q]
#             plt.plot(df_q['wavelength'], df_q['mean_reflectance'] * 100, label=f"Q{q}")  # reflectance as %

#         plt.title(f"{site_name} – Transect {transect}")
#         plt.xlabel("Wavelength (nm)")
#         plt.ylabel("Reflectance (%)")
#         plt.legend()
#         plt.grid(True)
#         plt.tight_layout()

#         if save:
#             filename = os.path.join(output_dir, f"{site_name}_Transect_{transect}.{file_format}")
#             plt.savefig(filename)
#             print(f"Saved: {filename}")

#         if show:
#             plt.show()
#         else:
#             plt.close()


In [ ]:
# plot_all_transects_per_site(summary_df, site_name="LCDM", show=False, save=True)


### Plot the stats by transect.

In [ ]:
#Plot stats by transect. 
import matplotlib.pyplot as plt

def plot_transect_with_stats(summary_df, site_name, show=True, save=False, output_dir=".", file_format="png"):
    """
    For each transect in a site:
    - Plot mean reflectance ± 1 SD
    - Plot CV (%) across wavelengths
    
    Parameters:
        summary_df (pd.DataFrame): Output of summarize_in_situ_data()
        site_name (str): e.g., "LCDM", "SD", "PV"
        show (bool): Show plots
        save (bool): Save plots
        output_dir (str): Where to save plots
        file_format (str): File format for saving
    """
    site_df = summary_df[summary_df["site"] == site_name]
    transects = sorted(site_df["transect"].unique())

    for t in transects:
        transect_df = site_df[site_df["transect"] == t]

        # Group across all quadrats in the transect
        grouped = (
            transect_df
            .groupby("wavelength")
            .agg(
                mean_reflectance=("mean_reflectance", "mean"),
                std_reflectance=("std_reflectance", "mean"),  # avg std across quadrats
                mean_cv=("cv_percent", "mean")  # avg CV across quadrats
            )
            .reset_index()
        )

        # Plot: Mean ± SD and CV%
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, height_ratios=[3, 1])

        # Top: Mean Reflectance ± SD
        ax1.plot(grouped["wavelength"], grouped["mean_reflectance"], color="blue", label="Mean Reflectance")
        ax1.fill_between(
            grouped["wavelength"],
            grouped["mean_reflectance"] - grouped["std_reflectance"],
            grouped["mean_reflectance"] + grouped["std_reflectance"],
            alpha=0.3,
            color="blue",
            label="±1 SD"
        )
        ax1.set_ylabel("Reflectance")
        ax1.set_title(f"{site_name} – Transect {t}")
        ax1.grid(True)
        ax1.legend()

        # Bottom: CV%
        ax2.plot(grouped["wavelength"], grouped["mean_cv"], color="red", label="Mean CV%")
        ax2.axhline(20, color="gray", linestyle="--", linewidth=0.8, label="20% Threshold")
        ax2.set_ylabel("CV (%)")
        ax2.set_xlabel("Wavelength (nm)")
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()

        # Save or show
        if save:
            filename = f"{site_name}_Transect{t}_stats.{file_format}"
            plt.savefig(f"{output_dir}/{filename}", dpi=300)
        if show:
            plt.show()
        else:
            plt.close()


In [ ]:
# Just display all quadrats in LCDM
plot_transect_with_stats(LCDM_summary, "LCDM")

# Save SD site quadrat plots as PNGs
plot_transect_with_stats(SD_summary, "SD")

# Save PV site quadrat plots as PDFs
plot_transect_with_stats(PV_summary, "PV")

### Optional: Download Transect Stats

In [ ]:
# from matplotlib.backends.backend_pdf import PdfPages
# import matplotlib.pyplot as plt

# def save_transect_stats_to_pdf(summary_df, site_name, output_path="transect_stats.pdf"):
#     """
#     Saves one plot per transect showing:
#     - Mean reflectance ± 1 SD
#     - CV (%) across wavelengths
#     into a multi-page PDF.
#     """
#     site_df = summary_df[summary_df["site"] == site_name]
#     transects = sorted(site_df["transect"].unique())

#     with PdfPages(output_path) as pdf:
#         for t in transects:
#             transect_df = site_df[site_df["transect"] == t]

#             grouped = (
#                 transect_df
#                 .groupby("wavelength")
#                 .agg(
#                     mean_reflectance=("mean_reflectance", "mean"),
#                     std_reflectance=("std_reflectance", "mean"),
#                     mean_cv=("cv_percent", "mean")
#                 )
#                 .reset_index()
#             )

#             fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, height_ratios=[3, 1])

#             ax1.plot(grouped["wavelength"], grouped["mean_reflectance"], color="blue", label="Mean Reflectance")
#             ax1.fill_between(
#                 grouped["wavelength"],
#                 grouped["mean_reflectance"] - grouped["std_reflectance"],
#                 grouped["mean_reflectance"] + grouped["std_reflectance"],
#                 alpha=0.3, color="blue", label="±1 SD"
#             )
#             ax1.set_ylabel("Reflectance")
#             ax1.set_title(f"{site_name} – Transect {int(t)}")
#             ax1.grid(True)
#             ax1.legend()

#             ax2.plot(grouped["wavelength"], grouped["mean_cv"], color="red", label="Mean CV%")
#             ax2.axhline(20, color="gray", linestyle="--", linewidth=0.8, label="20% Threshold")
#             ax2.set_ylabel("CV (%)")
#             ax2.set_xlabel("Wavelength (nm)")
#             ax2.grid(True)
#             ax2.legend()

#             plt.tight_layout()
#             pdf.savefig(fig)
#             plt.close(fig)


# save_transect_stats_to_pdf(LCDM_summary, "LCDM",
#     output_path=".../IntertidalOutputs/LCDM_transect_stats.pdf")

# save_transect_stats_to_pdf(SD_summary, "SD",
#     output_path=".../IntertidalOutputs/SD_transect_stats.pdf")

# save_transect_stats_to_pdf(PV_summary, "PV",
#     output_path=".../IntertidalOutputs/PV_transect_stats.pdf")


### Plot the stats for each quadrat

The code below graphs out each quadrat per site, not just the transects.

In [ ]:
#Plot stats by quadrat
import matplotlib.pyplot as plt

def plot_quadrats_with_stats(summary_df, site_name, show=True, save=False, output_dir=".", file_format="png"):
    """
    Plots one figure per quadrat in a site showing:
    - Mean reflectance ± 1 SD (top)
    - CV (%) (bottom)
    
    Parameters:
        summary_df (pd.DataFrame): Output of summarize_in_situ_data()
        site_name (str): e.g., "LCDM", "SD", "PV"
        show (bool): Show the plots in notebook
        save (bool): Save plots to file
        output_dir (str): Where to save files (if save=True)
        file_format (str): "png", "pdf", etc.
    """
    site_df = summary_df[summary_df["site"] == site_name]
    quadrats = sorted(site_df["quadrat"].unique())

    for q in quadrats:
        q_df = site_df[site_df["quadrat"] == q]

        if q_df.empty:
            continue

        # Aggregate just in case multiple wavelengths per quadrat
        grouped = (
            q_df
            .groupby("wavelength")
            .agg(
                mean_reflectance=("mean_reflectance", "mean"),
                std_reflectance=("std_reflectance", "mean"),
                mean_cv=("cv_percent", "mean")
            )
            .reset_index()
        )

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, height_ratios=[3, 1])

        # Top subplot: Reflectance ± SD
        # Convert to percentage
        mean_pct = grouped["mean_reflectance"] * 100
        std_pct = grouped["std_reflectance"] * 100

        # Top subplot: Reflectance ± SD (in %)
        ax1.plot(grouped["wavelength"], mean_pct, label="Mean Reflectance (%)", color='blue')
        ax1.fill_between(
            grouped["wavelength"],
            mean_pct - std_pct,
            mean_pct + std_pct,
            alpha=0.3,
            color='blue',
            label="±1 SD (%)"
        )
        ax1.set_ylabel("Reflectance (%)")
        transect_number = q_df["transect"].iloc[0]  # from data
        ax1.set_title(f"{site_name} – Transect {transect_number}, Quadrat {q}")
        ax1.grid(True)
        ax1.legend()

        # Bottom subplot: CV %
        ax2.plot(grouped["wavelength"], grouped["mean_cv"], label="CV%", color='red')
        ax2.axhline(20, linestyle="--", color="gray", linewidth=0.8, label="20% Threshold")
        ax2.set_ylabel("CV (%)")
        ax2.set_xlabel("Wavelength (nm)")
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()

        if save:
            filename = f"{site_name}_T{transect_number}_Q{q}.{file_format}"
            plt.savefig(f"{output_dir}/{filename}", dpi=300)
        if show:
            plt.show()
        else:
            plt.close()

            


In [ ]:
# Just display all quadrats in LCDM
plot_quadrats_with_stats(LCDM_summary, "LCDM")

# Save SD site quadrat plots as PNGs
plot_quadrats_with_stats(SD_summary, "SD")

# Save PV site quadrat plots as PDFs
plot_quadrats_with_stats(PV_summary, "PV")


### Optional: Download Quadrat Stats

In [ ]:
 #uncomment this code

# from matplotlib.backends.backend_pdf import PdfPages
# import matplotlib.pyplot as plt

# def save_all_quadrat_plots_to_pdf(summary_df, site_name, output_path="quadrat_summary.pdf"):
#     """
#     Saves all quadrat plots for a site into a single multi-page PDF.
#     Reflectance and standard deviation are plotted in percent.
#     """
#     site_df = summary_df[summary_df["site"] == site_name]
#     quadrats = sorted(site_df["quadrat"].unique())

#     with PdfPages(output_path) as pdf:
#         for q in quadrats:
#             q_df = site_df[site_df["quadrat"] == q]
#             if q_df.empty:
#                 continue

#             transect_number = q_df["transect"].iloc[0]

#             grouped = (
#                 q_df
#                 .groupby("wavelength")
#                 .agg(
#                     mean_reflectance=("mean_reflectance", "mean"),
#                     std_reflectance=("std_reflectance", "mean"),
#                     mean_cv=("cv_percent", "mean")
#                 )
#                 .reset_index()
#             )

#             # Convert mean and std reflectance to percentage
#             mean_pct = grouped["mean_reflectance"] * 100
#             std_pct = grouped["std_reflectance"] * 100

#             fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, height_ratios=[3, 1])

#             # Top: Mean Reflectance ± SD (in %)
#             ax1.plot(grouped["wavelength"], mean_pct, color="blue", label="Mean Reflectance (%)")
#             ax1.fill_between(
#                 grouped["wavelength"],
#                 mean_pct - std_pct,
#                 mean_pct + std_pct,
#                 color="blue",
#                 alpha=0.3,
#                 label="±1 SD (%)"
#             )
#             ax1.set_ylabel("Reflectance (%)")
#             ax1.set_title(f"{site_name} – Transect {transect_number}, Quadrat {q}")
#             ax1.grid(True)
#             ax1.legend()

#             # Bottom: CV %
#             ax2.plot(grouped["wavelength"], grouped["mean_cv"], color="red", label="CV%")
#             ax2.axhline(20, linestyle="--", color="gray", linewidth=0.8, label="20% Threshold")
#             ax2.set_ylabel("CV (%)")
#             ax2.set_xlabel("Wavelength (nm)")
#             ax2.grid(True)
#             ax2.legend()

#             plt.tight_layout()
#             pdf.savefig(fig)  # Save the figure to the PDF
#             plt.close(fig)    # Free memory

# # Save outputs

# save_all_quadrat_plots_to_pdf(LCDM_summary, "LCDM", output_path=".../IntertidalOutputs/LCDM_all_quadrats.pdf")
# save_all_quadrat_plots_to_pdf(SD_summary, "SD", output_path=".../IntertidalOutputs/SD_all_quadrats.pdf")
# save_all_quadrat_plots_to_pdf(PV_summary, "PV", output_path=".../IntertidalOutputs/PV_all_quadrats.pdf")



## Part 6: Extract and Plot EMIT Values at In Situ Quadrats. 

This part of the tutorial will go through loading in and utilizing EMIT and ECOSTRESS data for analyzation


### Process EMIT Data

Add file paths for EMIT RFL, this may be redundant, but can help for visualizations.

In [ ]:
import numpy as np
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from modules.emit_tools import emit_xarray

# LCDM
emit_fp_LCDM = "....../EMIT_L2A_RFL_001_20250525T213443_2514514_003.nc"
qa_fp_LCDM = "....../EMIT_L2A_MASK_001_20250525T213443_2514514_003.nc"

# PV
emit_fp_PV = ".../data/PV/EMIT_L2A_RFL_001_20250525T213443_2514514_003.nc"
qa_fp_PV = ".../data/PV/EMIT_L2A_MASK_001_20250525T213443_2514514_003.nc"

# SD
emit_fp_SD = ".../data/SD/EMIT_L2A_RFL_001_20250626T164730_2517711_009.nc"
qa_fp_SD = ".../data/SD/EMIT_L2A_MASK_001_20250626T164730_2517711_009.nc"


### FIX Part 6a: Filter EMIT Images 

UNCOMMENT OUT CLOUD MASK 
Load in your EMIT data and mask bad wavelengths to apply data cleaning. In addition, apply a cloud mask to remove cloudy pixels.

In [ ]:
def load_and_mask_emit(emit_fp, qa_fp):
    # Load EMIT reflectance
    emit_ds = emit_xarray(emit_fp, ortho=True)

    # Mask invalid wavelengths
    emit_ds['reflectance'].data[:, :, emit_ds['good_wavelengths'].data == 0] = np.nan
    emit_ds['reflectance'].data[emit_ds['reflectance'].data == -9999] = np.nan

    '''
      No cloud mask might be a problem, for this tutorial it is not necessary,
      but ensure that you uncomment the code below for analysis purposes if 
      commented out.
    '''
    
    # Load QA mask                                                        
    emit_mask = emit_xarray(qa_fp, ortho=True)
    emit_cloud_mask = emit_mask.sel(mask_bands='Aggregate Flag')
    emit_cloud_mask.mask.data[emit_cloud_mask.mask.data == -9999] = np.nan

    # Apply cloud mask
    emit_ds['reflectance'].data[emit_cloud_mask.mask.data == 1] = np.nan

    return emit_ds


In [ ]:
#Use this code to check the dimensions and coordinates of the emit data. 
# emit_mask = emit_xarray(qa_fp_PV, ortho=True) #change with variable name for different site.

# # Now inspect structure
# print("dims:", emit_mask.dims)
# print("coords:", emit_mask.coords)
# print("data_vars:", emit_mask.data_vars)
# emit_mask


In [ ]:
#Run the code
emit_ds_LCDM = load_and_mask_emit(emit_fp_LCDM, qa_fp_LCDM)
emit_ds_PV = load_and_mask_emit(emit_fp_PV, qa_fp_PV)
emit_ds_SD = load_and_mask_emit(emit_fp_SD, qa_fp_SD)

Plot the center of the spectra. If there is no spectra found, run the code to find the nearest point with data.

In [ ]:
#Use this plot to explore spectra if none is present at the center of the pixel. 
wavelength_to_plot = 850  # nm

layer_LCDM = emit_ds_LCDM.sel(wavelengths=wavelength_to_plot, method='nearest')
layer_PV = emit_ds_PV.sel(wavelengths=wavelength_to_plot, method='nearest')
layer_SD = emit_ds_SD.sel(wavelengths=wavelength_to_plot, method='nearest')


### Part 6b: Plot Filtered Data

Plot the different sites using the code below for the filtered EMIT images from Part 6a

In [ ]:
def gamma_adjust(rgb_ds, gamma=2.2, white_background=True):
    """
    Applies gamma correction to an RGB xarray Dataset.
    
    Parameters:
    - rgb_ds: xarray.Dataset with 'wavelengths' as the third dimension (should have shape [..., 3])
    - gamma: gamma value (default 2.2)
    - white_background: if True, clips reflectance to [0, 1] before applying gamma

    Returns:
    - xarray.Dataset with gamma-corrected RGB values
    """
    # Clip reflectance to valid range if desired
    rgb = rgb_ds.clip(0, 1) if white_background else rgb_ds

    # Apply gamma correction
    corrected = rgb.copy()
    corrected['reflectance'].data = np.power(rgb['reflectance'].data, 1 / gamma)

    return corrected


Create a geo-referenced image plot of a single EMIT layer. 

In [ ]:
def plot_site_layer(layer, site_name):
    return layer.hvplot.image(
        cmap='viridis',
        geo=True,
        tiles='ESRI',
        crs='EPSG:4326',
        frame_width=400,
        frame_height=400,
        xlabel='Longitude',
        ylabel='Latitude',
        title=f"{site_name}: {layer.wavelengths.values:.2f} nm"
    )


Set bands at specific wavelengths (650 nm, 560 nm, and 470 nm) to construct a true-color RGB image — just like what our eyes would see — using reflectance data from the EMIT satellite.

Why these wavelengths?
- 650 nm → Red
- 560 nm → Green
- 470 nm → Blue

These match the visible light spectrum:
- Red (R): 620–750 nm
- Green (G): 495–570 nm
- Blue (B): 450–495 nm

By selecting these specific bands, you can reconstruct a natural-looking image of the scene, where vegetation appears green, water appears dark, and urban areas may appear gray or tan — just like a regular satellite image.

In [ ]:
emit_rgb_LCDM = emit_ds_LCDM.sel(wavelengths=[650, 560, 470], method='nearest')
emit_rgb_PV   = emit_ds_PV.sel(wavelengths=[650, 560, 470], method='nearest')
emit_rgb_SD   = emit_ds_SD.sel(wavelengths=[650, 560, 470], method='nearest')

Run gamma_adjsut code to transform your RGB dataset so that it:

- looks visually better 
- is normalized for plotting 
- has enhanced visibility on light or white backgrounds

In [ ]:
emit_rgb_LCDM = gamma_adjust(emit_rgb_LCDM, gamma=0.5)
emit_rgb_PV   = gamma_adjust(emit_rgb_PV, gamma=0.5)
emit_rgb_SD   = gamma_adjust(emit_rgb_SD, gamma=0.5)

In [ ]:
plot_LCDM = plot_site_layer(layer_LCDM, "LCDM")
plot_PV = plot_site_layer(layer_PV, "PV")
plot_SD = plot_site_layer(layer_SD, "SD")


Plot sites separately, to engage with interactive maps individually.

In [ ]:
#Little Corona Del Mar / Newport Beach Plot
plot_LCDM


In [ ]:
#Palos Verdes Plot
plot_PV

In [ ]:
#San Diego Plot
plot_SD

### Part 6c: Plot Data on Interactive Map

Define function for interactive maps

In [ ]:
def build_interactive_view(emit_ds, emit_rgb, point_limit=10):
    import holoviews as hv
    from holoviews.streams import PointDraw, PointerXY, Tap
    import hvplot.xarray

    hv.extension('bokeh')
    color_cycle = hv.Cycle('Category20')

    # Create RGB map
    rgb_map = emit_rgb.hvplot.rgb(fontscale=1.5, xlabel='Longitude', ylabel='Latitude',
                                  frame_width=480, frame_height=480)

    # Midpoints
    xmid = emit_ds.longitude.values[int(len(emit_ds.longitude) / 2)]
    ymid = emit_ds.latitude.values[int(len(emit_ds.latitude) / 2)]

    # Set up point drawing
    first_point = ([xmid], [ymid], [0])
    points = hv.Points(first_point, vdims='id')
    points_stream = PointDraw(
        data=points.columns(),
        source=points,
        drag=True,
        num_objects=point_limit,
        styles={'fill_color': color_cycle.values[1:point_limit+1], 'line_color': 'gray'}
    )

    posxy = PointerXY(source=rgb_map, x=xmid, y=ymid)
    clickxy = Tap(source=rgb_map, x=xmid, y=ymid)

    # Dynamic map on click
    def click_spectra(data):
        coordinates = []
        if data is None or not any(len(d) for d in data.values()):
            coordinates.append((xmid, ymid))
        else:
            coordinates = [c for c in zip(data['x'], data['y'])]
        
        plots = []
        for i, coords in enumerate(coordinates):
            x, y = coords
            data_sel = emit_ds.sel(longitude=x, latitude=y, method="nearest")
            plots.append(
                data_sel.hvplot.line(
                    y="reflectance", x="wavelengths",
                    color=color_cycle, label=f"{i}"
                )
            )
            points_stream.data["id"][i] = i
        return hv.Overlay(plots)

    # Dynamic map on hover
    def hover_spectra(x, y):
        return emit_ds.sel(longitude=x, latitude=y, method='nearest').hvplot.line(
            y='reflectance', x='wavelengths', color='black', frame_width=400)

    click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])
    hover_dmap = hv.DynamicMap(hover_spectra, streams=[posxy])

    # Layout
    layout = hv.Layout(hover_dmap * click_dmap + rgb_map * points).cols(2).opts(
        hv.opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
        hv.opts.Overlay(show_legend=False, show_title=False, fontscale=1.5, frame_height=480)
    )
    return layout


Use the function to plot the data on respective interactive maps.

In [ ]:
layout_LCDM_interactive = build_interactive_view(emit_ds_LCDM, emit_rgb_LCDM)
layout_PV_interactive = build_interactive_view(emit_ds_PV, emit_rgb_PV)
layout_SD_interactive = build_interactive_view(emit_ds_SD, emit_rgb_SD)


Plot each map separately so analysis can be done individually.

In [ ]:
layout_LCDM_interactive

In [ ]:
layout_PV_interactive

In [ ]:
layout_SD_interactive

### Part 6d: Cropping EMIT to a Region of Interest

We only need to find relevant EMIT pixels to analyze.

In [ ]:
import hvplot.pandas
import geopandas as gp

def crop_and_plot_emit_site(site_name, emit_ds, shape_path, wavelength=850):
    """
    Crops the EMIT dataset to a region of interest and plots the result for a given site.

    Parameters:
        site_name (str): Site name (e.g., 'LCDM')
        emit_ds (xarray.Dataset): The full EMIT dataset
        shape_path (str): Path to the site's GeoJSON shape
        wavelength (int): Wavelength to visualize (default is 850nm)
    """
    shape = gp.read_file(shape_path)

    # Ensure CRS match
    if shape.crs != emit_ds.rio.crs:
        shape = shape.to_crs(emit_ds.rio.crs)

    # Clip EMIT to shape
    emit_cropped = emit_ds.rio.clip(shape.geometry.values, shape.crs, all_touched=True)

    # Create raster plot with transparency
    emit_plot = emit_cropped.sel(wavelengths=wavelength, method='nearest').hvplot.image(
        cmap='viridis', tiles='ESRI', crs='EPSG:4326', 
        alpha=0.6,  # Add transparency here
        title=f"{site_name} EMIT Reflectance @ {wavelength}nm",
        frame_width=400, frame_height=400, xlabel="Longitude", ylabel="Latitude"
    )

    # Plot quadrat points
    quadrat_plot = shape.hvplot.points(
        color='red', size=8, alpha=1, marker='circle',
        hover_cols=['Latitude', 'Longitude'] if 'Latitude' in shape.columns else None,
        crs='EPSG:4326'
    )

    return emit_plot * quadrat_plot


Assign variables to plot EMIT ROI with 

In [ ]:
plotROI_LCDM = crop_and_plot_emit_site("LCDM", emit_ds_LCDM, "....../LCDMQuadrat.geojson")
plotROI_PV   = crop_and_plot_emit_site("PV",   emit_ds_PV,   "....../PVQuadrat.geojson")
plotROI_SD   = crop_and_plot_emit_site("SD",   emit_ds_SD,   "....../SDQuadrat.geojson")

In [ ]:
plotROI_LCDM


In [ ]:
plotROI_PV



In [ ]:
plotROI_SD

## Part 7: Analyzing EMIT Spectra Data 

Now we know where our EMIT images are in relation to our in situ data. In order to conduct an analysis on the differences between the spectra, we need to graph the EMIT spectra. For this, we will plot the spectra only for the pixels that intersect with quadrat points. 

In [ ]:
import holoviews as hv
import numpy as np

def process_site_per_point(emit_ds, site_gdf, site_name):
    """
    Plots EMIT reflectance spectra for each point in the site GeoJSON.
    
    Parameters:
        emit_ds: xarray Dataset with EMIT data
        site_gdf: GeoDataFrame with points of interest (e.g., quadrat centers)
        site_name: Name of the site for labeling
        
    Returns:
        Holoviews Layout of line plots (1 per point)
    """
    # Clip EMIT to polygon shape
    print(f"Clipping EMIT data to {site_name}...")
    clipped = emit_ds.rio.clip(site_gdf.geometry.values, site_gdf.crs, all_touched=True)

    # Convert lat/lon from site_gdf to a list of coordinates
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]

    plots = []

    for idx, (lat, lon) in enumerate(coords):
        try:
            # Select nearest reflectance spectrum from EMIT data
            point_data = clipped.sel(latitude=lat, longitude=lon, method="nearest")

            # If reflectance is all NaNs, skip
            if np.isnan(point_data.reflectance.values).all():
                print(f"Skipping point {idx}: No valid reflectance data.")
                continue

            # Plot reflectance vs wavelength
            plot = point_data.hvplot.line(
                y='reflectance',
                x='wavelengths',
                color='black'
            ).opts(
                title=f"{site_name} Point {idx+1}\nLat={round(lat, 5)}, Lon={round(lon, 5)}",
                xlabel="Wavelength (nm)",
                ylabel="Reflectance",
                fontscale=1.2,
                width=400,
                height=250
            )
            plots.append(plot)
            print(f"✔️ Processed point {idx+1} at ({lat:.5f}, {lon:.5f})")

        except Exception as e:
            print(f"Error processing point {idx+1} at ({lat:.5f}, {lon:.5f}): {e}")

    if not plots:
        print(f"No valid spectra extracted for {site_name}.")
        return hv.Layout([])

    return hv.Layout(plots).cols(2)


In [ ]:
import geopandas as gpd

# Load the GeoJSON as a GeoDataFrame
site_gdf_LCDM = gpd.read_file("....../LCDMEMITQuadrat.geojson") #Change path depending on site you want to focus on. 

# Run the function with the correct input type
layout = process_site_per_point(emit_ds_LCDM, site_gdf_LCDM, "LCDM Site")
layout  # This will display the interactive layout in a notebook



Export the EMIT Spectra above for all the points. Only wavelengths between 400-900 nm will be saved, but can be changed based on desired output. 

In [ ]:
def EMITSpectra_to_pdf(emit_ds, site_gdf, site_name, output_path):
    """
    Extracts reflectance spectra for each point and saves styled plots to a PDF (400–900 nm range).

    Parameters:
        emit_ds: xarray.Dataset with 'reflectance' and 'wavelengths'
        site_gdf: GeoDataFrame of point geometries
        site_name: string, for labeling
        output_path: string, full path to output PDF
    """
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]
    wavelengths = emit_ds['wavelengths'].values

    with PdfPages(output_path) as pdf:
        for idx, (lat, lon) in enumerate(coords):
            try:
                point_data = emit_ds.sel(latitude=lat, longitude=lon, method='nearest')
                reflectance = point_data.reflectance.values

                if np.isnan(reflectance).all():
                    print(f"Skipping Point {idx+1}: No valid reflectance.")
                    continue

                # Filter for 400–900 nm
                mask = (wavelengths >= 400) & (wavelengths <= 900)
                filtered_wavelengths = wavelengths[mask]
                filtered_reflectance = reflectance[mask]

                # Plot
                fig, ax = plt.subplots(figsize=(4.5, 3.2))
                ax.plot(filtered_wavelengths, filtered_reflectance, color='black', linewidth=1)

                # Title and subtitle
                ax.set_title(f"{site_name} Point {idx+1}\nLat={lat:.5f}, Lon={lon:.5f}",
                             fontsize=10, fontweight='bold', loc='center', pad=10)

                # Axes
                ax.set_xlabel("Wavelength (nm)", fontsize=9, style='italic', labelpad=5)
                ax.set_ylabel("Reflectance", fontsize=9, style='italic', labelpad=5)
                ax.set_xlim(400, 900)
                ax.set_ylim(bottom=0)

                # Formatting
                ax.tick_params(labelsize=8)
                ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.4)

                plt.tight_layout()
                pdf.savefig(fig)
                plt.close(fig)

                print(f"✔️ Saved Point {idx+1} to PDF")

            except Exception as e:
                print(f"Error at Point {idx+1} ({lat:.5f}, {lon:.5f}): {e}")

    print(f"\nPDF successfully saved to:\n{output_path}")

EMITSpectra_to_pdf(
    emit_ds=emit_ds_LCDM,
    site_gdf=site_gdf_LCDM,
    site_name="LCDM Site",
    output_path=".../LCDM_Spectra_400-900nm.pdf"
)



Plot and save the EMIT reflectances by transect. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def compute_emit_stats_by_transect(emit_ds, site_gdf, site_name="LCDM", group_col="transect", save_prefix=None):
    """
    Computes mean and std reflectance per transect from EMIT data.
    Plots mean reflectance with shaded std bands for each transect.
    
    Returns:
        stats_df: DataFrame with ['transect', 'wavelength', 'mean', 'std']
    """
    wavelengths = emit_ds['wavelengths'].values
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]
    transects = site_gdf[group_col].values

    grouped_data = defaultdict(list)

    for idx, (lat, lon, transect) in enumerate(zip(*zip(*coords), transects)):
        try:
            point_data = emit_ds.sel(latitude=lat, longitude=lon, method="nearest")
            reflectance = point_data.reflectance.values

            if np.isnan(reflectance).all():
                continue

            grouped_data[transect].append(reflectance)

        except Exception as e:
            print(f"Skipping point {idx+1}: {e}")

    all_stats = []

    for transect, spectra in grouped_data.items():
        arr = np.stack(spectra)
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)

        for i, wl in enumerate(wavelengths):
            if 400 <= wl <= 900:
                all_stats.append({
                    "transect": transect,
                    "wavelength": wl,
                    "mean": mean[i],
                    "std": std[i]
                })

    stats_df = pd.DataFrame(all_stats)

    # === Plot Mean with ± Std Error Shading ===
    fig, ax = plt.subplots(figsize=(12, 7))

    for transect in sorted(stats_df['transect'].unique()):
        sub = stats_df[stats_df['transect'] == transect]
        ax.plot(sub['wavelength'], sub['mean'], label=f'Transect {transect}')
        ax.fill_between(sub['wavelength'], sub['mean'] - sub['std'], sub['mean'] + sub['std'], alpha=0.2)

    ax.set_title(f"{site_name} Site – Mean EMIT Reflectance per Transect")
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Reflectance")
    ax.legend(title="Transect", loc='upper left', ncol=2)
    ax.grid(True)
    plt.tight_layout()

    if save_prefix:
        pdf_path = f"{save_prefix}_Mean_STD_Reflectance.pdf"
        plt.savefig(pdf_path, dpi=300)
        print(f"Plot saved to {pdf_path}")

    plt.show()
    plt.close()

    return stats_df


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.backends.backend_pdf import PdfPages

def compute_emit_stats_by_transect(emit_ds, site_gdf, site_name="LCDM", group_col="transect", save_prefix=None):
    """
    Computes mean, std, and cv reflectance per transect from EMIT data.
    Plots all metrics in one figure (3 subplots) and saves to a single-page PDF.

    Returns:
        stats_df: DataFrame with ['transect', 'wavelength', 'mean', 'std', 'cv']
    """
    wavelengths = emit_ds['wavelengths'].values
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]
    transects = site_gdf[group_col].values

    grouped_data = defaultdict(list)

    for idx, (lat, lon, transect) in enumerate(zip(*zip(*coords), transects)):
        try:
            point_data = emit_ds.sel(latitude=lat, longitude=lon, method="nearest")
            reflectance = point_data.reflectance.values

            if np.isnan(reflectance).all():
                continue

            grouped_data[transect].append(reflectance)

        except Exception as e:
            print(f"Skipping point {idx+1}: {e}")

    all_stats = []

    for transect, spectra in grouped_data.items():
        arr = np.stack(spectra)
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        cv = std / mean

        for i, wl in enumerate(wavelengths):
            if 400 <= wl <= 900:
                all_stats.append({
                    "transect": transect,
                    "wavelength": wl,
                    "mean": mean[i],
                    "std": std[i],
                    "cv": cv[i]
                })

    stats_df = pd.DataFrame(all_stats)

    # Plot all metrics in one figure with 3 subplots
    fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

    for metric, ax in zip(['mean', 'std', 'cv'], axes):
        for transect in sorted(stats_df['transect'].unique()):
            sub = stats_df[stats_df['transect'] == transect]
            ax.plot(sub['wavelength'], sub[metric], label=f'Transect {transect}')
        ax.set_ylabel(metric.upper())
        ax.set_title(f"{site_name} – {metric.upper()} EMIT Reflectance per Transect")
        ax.grid(True)
        ax.legend(loc='upper right')

    axes[-1].set_xlabel("Wavelength (nm)")
    plt.tight_layout()

    # Save as PDF if requested
    if save_prefix:
        pdf_path = f"{save_prefix}_EMIT_Transect_Reflectance.pdf"
        plt.savefig(pdf_path)
        print(f"Plot saved to {pdf_path}")

    plt.show()
    plt.close()

    return stats_df


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.backends.backend_pdf import PdfPages

def compute_emit_stats_by_transect(emit_ds, site_gdf, site_name="LCDM", group_col="transect", save_prefix=None):
    """
    Computes mean and std reflectance per transect from EMIT data.
    Plots both metrics in a single figure (2 subplots) and optionally saves as PDF.

    Returns:
        stats_df: DataFrame with ['transect', 'wavelength', 'mean', 'std']
    """
    wavelengths = emit_ds['wavelengths'].values
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]
    transects = site_gdf[group_col].values

    grouped_data = defaultdict(list)

    for idx, (lat, lon, transect) in enumerate(zip(*zip(*coords), transects)):
        try:
            point_data = emit_ds.sel(latitude=lat, longitude=lon, method="nearest")
            reflectance = point_data.reflectance.values

            if np.isnan(reflectance).all():
                continue

            grouped_data[transect].append(reflectance)

        except Exception as e:
            print(f"⚠️ Skipping point {idx+1}: {e}")

    all_stats = []

    for transect, spectra in grouped_data.items():
        arr = np.stack(spectra)
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)

        for i, wl in enumerate(wavelengths):
            if 400 <= wl <= 900:
                all_stats.append({
                    "transect": transect,
                    "wavelength": wl,
                    "mean": mean[i],
                    "std": std[i]
                })

    stats_df = pd.DataFrame(all_stats)

    # Plot mean and std in two subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    for metric, ax in zip(['mean', 'std'], axes):
        for transect in sorted(stats_df['transect'].unique()):
            sub = stats_df[stats_df['transect'] == transect]
            ax.plot(sub['wavelength'], sub[metric], label=f'Transect {transect}')
        ax.set_ylabel(metric.upper())
        ax.set_title(f"{site_name} – {metric.upper()} EMIT Reflectance per Transect")
        ax.grid(True)
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)

    axes[-1].set_xlabel("Wavelength (nm)")
    plt.tight_layout(pad=2.0)

    if save_prefix:
        pdf_path = f"{save_prefix}_EMIT_Transect_Reflectance.pdf"
        plt.savefig(pdf_path, bbox_inches='tight')  # Ensures legend doesn't get cut off
        print(f"Plot saved to {pdf_path}")

    plt.show()
    plt.close()

    return stats_df


In [ ]:
df_stats = compute_emit_stats_by_transect(
    emit_ds=emit_ds_LCDM,
    site_gdf=site_gdf_LCDM,
    site_name="LCDM Site",
    group_col="Transect#",  # <-- match the actual column name
    save_prefix="LCDM_Reflectance"
)


## Part 8: Export EMIT Data Points 

While you can visualize this on Python, it can be easier to revise and manipulate your maps on a mapping software such as QGIS or ArcGIS. The code below downloads the points that were assigned with the spectra above. 

In [ ]:
def extract_reflectance_table(emit_ds, site_gdf, site_name, output_fp):
    """
    Extracts reflectance spectra for each point and saves to a CSV/GeoPackage.

    Parameters:
        emit_ds: xarray Dataset (EMIT)
        site_gdf: GeoDataFrame of points
        site_name: string name
        output_fp: path to save CSV or GeoPackage
    """
    coords = [(geom.y, geom.x) for geom in site_gdf.geometry]
    wavelengths = emit_ds['wavelengths'].values
    records = []

    for idx, (lat, lon) in enumerate(coords):
        try:
            point_data = emit_ds.sel(latitude=lat, longitude=lon, method="nearest")

            refl = point_data.reflectance.values
            if np.isnan(refl).all():
                continue

            record = {
                'site': site_name,
                'point_id': idx + 1,
                'latitude': lat,
                'longitude': lon,
                **{f"{int(wl)}nm": refl[i] for i, wl in enumerate(wavelengths)}
            }
            records.append(record)

        except Exception as e:
            print(f"Skipping point {idx+1}: {e}")

    # Turn into DataFrame
    df = pd.DataFrame(records)

    # Optionally make it a GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['longitude'], df['latitude']),
        crs=site_gdf.crs
    )

    # Save
    if output_fp.endswith(".gpkg"):
        gdf.to_file(output_fp, driver="GPKG")
    else:
        gdf.drop(columns="geometry").to_csv(output_fp, index=False)

    print(f"Saved reflectance table to {output_fp}")


Example usage below, change acronym "LCDM" with respective site name. 

In [ ]:
extract_reflectance_table(
    emit_ds_LCDM,
    site_gdf_LCDM,
    "LCDM",
    output_fp=".../LCDM_reflectance.gpkg" #Change to desired filepath and geopackage name 
)


In [ ]:
import matplotlib.pyplot as plt

# Create an empty plot with defined x-axis range
plt.figure(figsize=(10, 6))
plt.xlabel("Wavelength (nm)")
plt.ylabel("Mean Reflectance (%)")
plt.title("LCDM")
plt.xlim(400, 900)  # Set x-axis range from 400 to 900
plt.ylim(0, 100)    # Optional: reflectance in percent (0–100)
plt.grid(True)
plt.tight_layout()
plt.show()
